In [1]:
from  ipyleaflet import (Map, 
                      #   GeoData, 
                         basemaps, 
                       #  WidgetControl, 
                       #  LayersControl, 
                         Marker) 

In [2]:
center = [57.71, 11.98]
zoom = 2
n = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=center, zoom=zoom)
marker = Marker(location=center, draggable=True)
n.add_layer(marker);
display(n)

Map(center=[57.71, 11.98], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…

In [4]:
print(marker.location)

[47.754097979680026, 35.85937500000001]


In [1]:
print('test')

test


In [20]:
%%bash
cd ../../pi_wrf/pi_wrf
ls
pwd
python3 ../../run_wrf

Pages
Runner.py
__init__.py
calculate_distance.py
color_schemes.py
map.pkl
set_domain.py
/pi-wrf/pi_wrf/pi_wrf


  File "../../run_wrf", line 10
    export wrf_basis=$(readlink -f $(dirname $(basename $0)))
                   ^
SyntaxError: invalid syntax


CalledProcessError: Command 'b'cd ../../pi_wrf/pi_wrf\nls\npwd\npython3 ../../run_wrf\n'' returned non-zero exit status 1

In [19]:
%%bash
cd ../..
ls
pwd

Output
WRF_System
pi_wrf
run_wrf
src
/pi-wrf
